In [ ]:
! pip install jamotools
! pip install g2pk

In [2]:
path = '/content/drive/MyDrive/Voice_Data/KoG2P/'  

In [3]:
import os
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook
from g2pk import G2p
g2p = G2p()
import jamotools
import time

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [4]:
def mkdf(path):
    df = pd.read_csv(path, delimiter=' :: ', header=None) 
    df.columns=['path', 'txt']
    return df 

def bracket_filter(sentence):
    new_sentence = str()
    flag = False
    
    for ch in sentence:
        if ch == '(' and flag == False: 
            flag = True
            continue
        if ch == '(' and flag == True:
            flag = False
            continue
        if ch != ')' and flag == False:
            new_sentence += ch
    return new_sentence

def special_filter(sentence):
    
    try:
        import re
    except ImportError:
        print("pip install re")
        
    SENTENCE_MARK = ['?', '!']
    NOISE = ['o', 'n', 'u', 'b', 'l']
    EXCEPT = ['/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '=', ':', ';', '.', ',']
    
    new_sentence = str()
    for idx, ch in enumerate(sentence):
        if ch not in SENTENCE_MARK:
            # o/, n/ 등 처리
            if idx + 1 < len(sentence) and ch in NOISE and sentence[idx+1] == '/': 
                continue 

        if ch == '#': 
            new_sentence += '샾'

        elif ch not in EXCEPT: 
            new_sentence += ch

    pattern = re.compile(r'\s\s+')
    new_sentence = re.sub(pattern, ' ', new_sentence.strip())
    return new_sentence

def sentence_filter(raw_sentence):
    return special_filter(bracket_filter(raw_sentence))
    

In [5]:
from glob import glob
txt = glob(path+'val_txt/*.txt')
txt

['/content/drive/MyDrive/Voice_Data/KoG2P/val_txt/broadcast_01_scripts.txt',
 '/content/drive/MyDrive/Voice_Data/KoG2P/val_txt/hobby_01_scripts.txt',
 '/content/drive/MyDrive/Voice_Data/KoG2P/val_txt/dialog_01_scripts.txt',
 '/content/drive/MyDrive/Voice_Data/KoG2P/val_txt/life_01_scripts.txt',
 '/content/drive/MyDrive/Voice_Data/KoG2P/val_txt/weather_01_scripts.txt',
 '/content/drive/MyDrive/Voice_Data/KoG2P/val_txt/economy_01_scripts.txt',
 '/content/drive/MyDrive/Voice_Data/KoG2P/val_txt/play_01_scripts.txt',
 '/content/drive/MyDrive/Voice_Data/KoG2P/val_txt/shopping_01_scripts.txt']

In [6]:
df_broadcast = mkdf(txt[0])
df_hobby = mkdf(txt[1])
df_dialog = mkdf(txt[2])
df_life = mkdf(txt[3])
df_weather = mkdf(txt[4])
df_economy = mkdf(txt[5])
df_play = mkdf(txt[6])
df_shopping = mkdf(txt[7])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [33]:
def val_pipe(df,name:str):

    path = '/content/drive/MyDrive/Voice_Data/KoG2P/'  

    os.mkdir(path+'txt_dataframe/val_{}'.format(name))
    os.chdir(path+'txt_dataframe/val_{}'.format(name))

    null_idx = df[df.txt.isna()].index
    pd.DataFrame(list(df[df.txt.isna()].path)).to_csv('val_{}_null_path.csv'.format(name), index=False)
    df.drop(null_idx, axis=0, inplace=True)

    df.txt = [sentence_filter(i) for i in df.txt]

    if df.txt.isna().sum() > 0:
        add_null_idx = df[df.txt.isna()].index
        pd.DataFrame(list(df[df.txt.isna()].path)).to_csv('val_{}_null_path2.csv'.format(name), index=False)

        df.drop(add_null_idx, axis=0, inplace=True)
    
    print('complete 1')

    df.txt = [g2p(i) for i in tqdm_notebook(df.txt)] 
    df.to_csv('val_{}_g2pk.csv'.format(name), index=False)

    print('complete 2')

    
    df.txt = [jamotools.split_syllables(u"{}".format(i)) for i in df.txt]
    df.to_csv('val_{}_jamo.csv'.format(name), index=False)

    print('complete 3')

    path_list = list(df.path) 
    txt_iist = list(df.txt)
    name_list = [i.split('/')[-1].split('.')[0] for i in df.path]
    
    df.path = name_list  
    df.columns=['name','txt']
    df.to_csv('val_{}_text.csv'.format(name), index=False)
    
    print('complete 4')

    df.txt = path_list
    df.columns=['name','path']
    df.to_csv('val_{}_wavscp.csv'.format(name), index=False)

    print('complete 5')

    spk = [i.split('/')[1].split('.')[1][0]+'_'+i.split('/')[-1].split('.')[0][0]+'_'+i.split('/')[-1].split('.')[0].split('_')[1] for i in df.path]
    df.path = spk
    df.columns=['name', 'spk']
    df.to_csv('val_{}_utt2spk.csv'.format(name), index=False)

    print('complete 6 ')
    
    df=df[['spk', 'name']]
    df.to_csv('val_{}_spk2utt.csv'.format(name), index=False)

    print('complete all')


In [ ]:
#val_pipe(df_broadcast,'broadcast')
#val_pipe(df_hobby, 'hobby')
val_pipe(df_dialog, 'dialog')
val_pipe(df_life, 'life')
val_pipe(df_weather, 'weather')
val_pipe(df_economy, 'economy')
val_pipe(df_play, 'play')
val_pipe(df_shopping, 'shopping')

complete 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



complete 2
complete 3
complete 4
complete 5
complete 6 
complete all
complete 1



complete 2
complete 3
complete 4
complete 5
complete 6 
complete all
complete 1



complete 2
complete 3
complete 4
complete 5
complete 6 
complete all
complete 1
